In [2]:
%matplotlib notebook
import IPython.display
import glob
# from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import virtualhome
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

In [3]:
# Connect to virtual home simulator

comm = UnityCommunication(timeout_wait=1000)


In [16]:
# Reset the scene and get the graph to scene 3
comm.reset(3)
s, g = comm.environment_graph()

In [17]:
# Add a agent 
comm.add_character('Chars/female2', initial_room='kitchen')

# Get nodes for apple, desk, kitchen
apple = [node['id'] for node in g['nodes'] if node['class_name'] == 'apple'][1]
desk = [node['id'] for node in g['nodes'] if node['class_name'] == 'desk'][1]
kitchen = [node['id'] for node in g['nodes'] if node['class_name'] == 'kitchen'][0]
fridge = [node['id'] for node in g['nodes'] if node['class_name'] == 'fridge'][0]
tv = [node['id'] for node in g['nodes'] if node['class_name'] == 'tv'][0]
table = [node['id'] for node in g['nodes'] if node['class_name'] == 'kitchentable'][0]
microwave = [node['id'] for node in g['nodes'] if node['class_name'] == 'microwave'][0]
faucet_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'faucet'][-1]


# get the light nodes in the scene
light_nodes = []

for light_node in [node for node in g['nodes'] if node['class_name'] == 'lightswitch']:
    light_nodes.append(light_node)
light_node = [x for x in g['nodes'] if x['class_name'] == 'lightswitch']

In [18]:
# Turn on all the lights in the house

script = [
        '<char0> [switchon] <lightswitch> ({})'.format(light_nodes[3]['id']),
        '<char0> [switchon] <lightswitch> ({})'.format(light_nodes[0]['id']),      
          '<char0> [switchon] <lightswitch> ({})'.format(light_nodes[1]['id']),
          '<char0> [switchon] <lightswitch> ({})'.format(light_nodes[2]['id']),
          # Move the apple to the desk
            '<char0> [grab] <apple> ({})'.format(apple), 
          '<char0> [put] <apple> ({}) <desk> ({})'.format(apple, desk),
            # open fridge
          '<char0> [walk] <fridge> ({})'.format(fridge),
          '<char0> [open] <fridge> ({})'.format(fridge),
            # close fridge
            '<char0> [close] <fridge> ({})'.format(fridge),

          # turn on faucet
          '<char0> [walk] <faucet> ({})'.format(faucet_id),
          '<char0> [switchon] <faucet> ({})'.format(faucet_id),
            # Turn on microwave
          '<char0> [walk] <microwave> ({})'.format(microwave),
          # Open microwave
            '<char0> [open] <microwave> ({})'.format(microwave),
            # close microwave
            '<char0> [close] <microwave> ({})'.format(microwave),
          '<char0> [switchon] <microwave> ({})'.format(microwave),

            '<char0> [walk] <tv> ({})'.format(tv),
          '<char0> [switchon] <tv> ({})'.format(tv),
          # walk to facuet
            '<char0> [walk] <faucet> ({})'.format(faucet_id),
            # turn off faucet
            '<char0> [switchoff] <faucet> ({})'.format(faucet_id),
            '<char0> [walk] <kitchentable> ({})'.format(table)

         ]

success, message = comm.render_script(script=script[0:20], 
                                    find_solution=True,
                                    processing_time_limit=200,
                                    frame_rate=15,
                                    image_width=640, image_height=480,
                                    # image_width=512, image_height=320,
                                    skip_animation=False,
                                    image_synthesis=['normal'],
                                    camera_mode=['PERSON_FROM_BACK'],
                                    recording=True)